# Projet d'étude Tests d'hypothèse
*Rédaction Rémi de Guiran. D'après une idée de Florian Dadouchi*

Pour les 3 journées de cours consacrées à cette partie, nous allons nous intéresser à la détection d'un signal sonore. Cela nous amènera en première itération à réaliser une étude sur le bruit de fond. Puis nous nous intéresserons à la détection d'un signal sonore présent au milieu du bruit de fond. Pour réaliser une détection, nous allons devoir faire une hypothèse qui nous permettra de décider si oui ou non, un signal sonore est détecté. 

Vous aborderez les notions suivantes :
* Fonctions de distribution en statistique (continues et discrètes)
* Elements de probabilités
* Tests d'hypothèses (choix d'une hypothèse nulle, d'une hypothèse alternative, critère de significativité et p-value)

Si cela remonte à loin, vous aurez besoin de vous raffraichir la mémoire sur certaines notions :
* Les bases du calcul différentiel et intégral (Qu'est ce qu'une dérivée ? Une intégrale ? etc). Vous n'aurez pas besoin de calculer des intégrales analytiquement mais il sera important de comprendre ces concepts pour avancer
* Les bases de probabilités.

**Sujet :** 
Depuis quelque temps, bien que vivant dans un endroit très calme la nuit, vous vous demandez si des sons ne vous dérangent pas pendant votre sommeil. Bien décidé à avoir le coeur net et plutôt que de vous demander à chaque reveil si celui-ci est dû à un son quelconque ou juste à une mauvaise qualité de sommeil, vous décidez de réaliser un enregistrement sonore la nuit, et de créer un test qui permettra de détecter les sons qui se seront produits durant cette nuit.

Tous les enregistrements sont disponibles avec le sujet, chacun en double :
* un au format ".wav" (ça serait dommage de ne pas utiliser vos oreilles alors que vous travaillez sur des signaux sonores)
* l'autre et en format .txt.

Chaque enregistrement est **échantillonné à 48 kHz**.

Pour éviter de perdre du temps, la variable "signal sonore" contenue dans chaque fichier .txt est déjà normalisée, pour avoir des amplitudes de signaux n'excédant pas quelques unités. Vous n'aurez donc pas besoin de modifier vos données pour travailler dessus.


**Information importante :**

Toutes les lois de probabilités porteront sur la valeur absolue de l’amplitude mesurée. De manière générale, par défaut toute référence à une amplitude renvoie à la valeur absolue de l’amplitude mesurée. Le cas contraire vous sera explicité. L’abréviation **va** renvoie à la valeur absolue, et **vr** à la valeur réelle.


**Resources :**


* Lois de probabilités :

Poly "Statistique Appliquée", chapitres 2.1 et 3.1

https://www.youtube.com/watch?v=YXLVjCKVP7U

https://statisticsbyjim.com/basics/probability-distributions/




* Tests d'hypothèses :

Poly "Statistique : Tests d'hypothèses", chapitre 1

https://www.youtube.com/watch?v=tTeMYuS87oU

https://towardsdatascience.com/everything-you-need-to-know-about-hypothesis-testing-part-i-4de9abebbc8a

# Iteration 1 - Etude du bruit et d'un son

# A Etude du bruit

Avant de commencer à réfléchir à un quelconque test de detection de son, vous décidez d'étudier le bruit de fond. Le bruit c'est en fait tout ce qui ne vous intéresse pas, une fluctuation très faiblement audible (écouter le fichier wav) et qui ne vous dérange pas pendant votre sommeil (vous vivez dans un endroit calme). Ce qui vous intéresse in fine, ce sont les sons (on distingue donc bien les notions de son et de bruit). Mais pour espérer les détecter des **sons**, il vous faut d'abord caractériser **le bruit**.

##  A.0 - Ecouter le fichier "bruit.wav" dans un lecteur audio

## A.1 - Importer le bruit "bruit_seul.txt" dans un array numpy

Vous pouvez utiliser la fonction loadtxt de numpy

In [1]:
import numpy as np

## A.2 - Plotter l'amplitude mesurée (en vr) en fonction du temps pour la première seconde d'enregistrement

On rappelle que la fréquence d'échantillonnage est 48 kHz

## A.3 - Afficher les probability density function (PDF) et  la cumulative distribution function (CDF) empiriques de l'amplitude du bruit en va

## A.4 - Identifier les 2 lois de probabilités obtenues à la question précédente 

### a - Pouvez-vous identifier les 2 fonctions ?

Au vu de la forme des 2 fonctions, à quelles fonction analytiques ressemblent-elles ? Si vous avez une idée précise, identifiez la et donnez ses coefficients caractéristiques.
Sinon rendez-vous question suivante :)

### b - Calculez la moyenne et l'écart type du bruit (pas en valeur absolue cette fois)
Info : la classe `nd_array` du module `numpy` possède des méthodes qui font ça directement.

### c - 1 - Sur le même graphe, affichez la PDF de $ \mid bruit \mid $ et la gaussienne de moyenne et d'écart type calculée question précédente

### c - 2 - Expliquez la différence entre les deux courbes.


## A.5 - Evaluer la probabilité des amplitudes du bruit pour différentes plages d'amplitudes

Ici comme très souvent on différencie le bruit du son. Le bruit est une fluctuation statistique, alors que le son est porteur d'information.

On va considérer dans la suite de cette itération que le bruit est parfaitement caractérisé par une loi demi-normale, de moyenne $\mu =0$ et d'écart type $\sigma = 1 $. Sa fonction de densité (PDF) est :
$$ f(x) = \frac{2}{\sigma \sqrt{2 \pi}} \mathrm{e}^{- \frac{1}{2} \left( \frac{ (x - \mu)^2}{\sigma^2} \right)  }$$

On utilisera donc pour répondre aux questions suivantes (A.5) les CDF et PDF analytiques (c'est à dire théoriques) puisque l'on fait l'hypothèse que notre bruit est régi par une distribution demi-normale.

Vous pouvez explorer la doc de scipy.stats pour trouver les fonctions associées à diverses distributions : https://docs.scipy.org/doc/scipy/tutorial/stats.html

### a -  Donner la formule analytique de la fonction de répartition (CDF) de la loi demi-normale
(Pour les plus matheux elle peut se déduire de la PDF, sinon wikipedia est votre ami).

### b - 1 Probabilités sur les amplitudes du bruit

* Quelle est la probabilité d'observer des amplitudes comprises dans $[0 , \sigma]$  ?
* Même question pour $[0 , 2\sigma]$
* Même question pour $[0, 3\sigma]$


### b - 2 Probabilités sur les amplitudes du bruit

À l'inverse on s'intéresse maintenant aux amplitudes comprises dans l'intervalle  $[a,\infty]$ (avec $a > 0$).

Déterminer les valeur de a pour que la probabilité de d'observer une amplitude supérieure à $a$ (donc comprise dans $[a,\infty]$) soit de :

- 5 %
- 1%
- 0.1%


# B - Etude d'un son

Vous avez réalisé un enregistrement pendant la nuit. En inspectant rapidement la courbe amplitude(temps), il vous semble voir quelque chose de particulier, et vous soupçonnez fortement qu'il ne s'agisse pas du bruit mais d'un son (une tentative d'effraction ?). Vous trouverez l'extrait en question dans le fichier "son.wav".

## B.0 -  Ecoutez l'extrait

Bon c'est effectivement un son, mais rien d'alarmant. Mais il n'empêche que vous aimerez pouvoir détecter les sons sans avoir à regarder le signal tous les matins.

## B.1 -  Importez le fichier "son.txt" dans un array numpy et faites un plot comparant un zone exclusivement bruitée et une zone de son

## B.2 -  Identifier approximativement la zone de son et afficher la PDF et la CDF empiriques de l'amplitude (en va), quelle différence avec celles du bruit ?

## B.3 - Afficher pour différents seuils d’amplitude (en va) les index où l’amplitude est supérieure à ce seuil (en plottant avec deux couleurs différentes par exemple). 

Conclure sur la possibilité d'identifier l'intervalle du son avec cette méthode.

# Itération 2 - Test d'hypothèse

Plutôt que de faire des tests sur les occurences ponctuelles d'amplitude supérieure à une valeur seuil, vous décidez de faire une étude similaire portant sur le nombre d'occurences durant un certain temps. De cette manière, si le bruit atteint ponctuellement une amplitude importante, on peut se douter qu'il a très peu de chance de réitérer cet "exploit" beaucoup de fois dans un intervalle de temps court. Tout l'enjeu sera de quantifier cette "chance" (ie cette probabilité).

# A - Un peu de proba


* On note une limite d'amplitude $a_l$. La probabilité d'obtenir sur une unique mesure (on peut dire tirage comme pour les jeux de hasard) une amplitude $a$, telle que $a \geq a_l $ est notée $p$.
* On note $n$, le nombre d'amplitudes mesurées sur un intervalle (échantillon).
* On note $k$ un entier positif inférieur ou égal à $n$. 



## A.1 - Donnez la formule de $P_k^{n}$, la probabilité de  tirer $k$ fois une amplitude supérieure ou égale à $a_l$ lors de $n$ tirages.

Les plus affamés d'entre vous peuvent la démontrer. Sinon une recherche sur internet suffira largement.




## A.2 - Trouvez dans un package python existant une fonction  qui permet de calculer cette probabilité $P_k^n$




## A.3 - Soit $n =20 $ :

### a - Pour plusieurs valeurs de $p$, calculez $P_k^{n}$ pour chaque $k=1..n$

Vérifiez bien que la somme des probabilités sur toutes les valeurs de $k$ pour chaque loi vaut 1

###  b - Affichez en une sur un plot en barre

### c - Affichez les toutes sur le même graphe
Conseil : pour la visibilité utilisez plutôt plot que bar

Question subsidiaire : quel objet mathématique permet d'unifier le concept de loi de probabilité discrète et celui de densité de probabilité qui s'applique aux variables aléatoire continues (pdf) ?

# B - Construction du test d'hypothèse

Vous voulez donc construire votre test d'hypothèse. On s'appuiera pour ce test sur la loi binomiale étudiée partie A. Pour cela, vous avez besoin de définir une hypothèse nulle, une hypothèse alternative, ainsi qu'un critère de significativité.

* Il faudra identifier l'amplitude $a_l$ correspondant à une probabilité $p$. Cette probabilité correspond à la probabilité, sur une seule mesure du bruit, d'obtenir une amplitude supérieure ou égale à $a_l$. Donc vous devrez décider d'abord de la valeur de $p$ pour en déduire $a_l$ (si vous êtes confus, revoyez la partie 4-b de l'itération 1).
* Vous aurez besoin de choisir un $n$. Vous comptabiliserez pour un intervalle comprenant $n$ mesures, le nombre $N_a$ de fois que vous aurez obtenu une mesure $a \geq a_l$.

Mais d'abord :

## B.1 - Questions théoriques (sans code)

### a - Caractérisez l'hypothèse nulle de votre test $H_{0} $



### b - Caractérisez l'hypothèse alternative $H_1$ (ou $H_a$ selon les gouts)



### c - Identifier l'élément mesurable (i.e. la statistique) qui permettra de réaliser le test


### d - Rappeler la loi de probabilité de cet élément mesurable (ie cette statistique "sous" $H_0$)


### e -  Définir la p-value d'un test (donner la formule), et la valeur qu'elle prend pour ce test en particulier

### f - Étant donné le seuil de significativité $\alpha$, identifier dans quelles situations $H_0$ sera rejetée, ou bien  non rejetée.

## B.2 Réaliser un test d'hypothèse :

Comme énoncé au début de cette partie B, il faut choisir une valeur pour $p$ et pour $n$. On se fixera $n = 100$, et $p = 0.01$. On se donnera également $\alpha = 0.05 $.

On commencera d'abord par travailler sur un intervalle de 100 mesures au milieu du signal sonore, entre les indices : $[505400, 505500]$.

### B.2.a - Plotter la PMF loi de probabilité de l'élément mesurable "sous" $H_0$

### B.2.b - Calculer la valeur de l'élément mesurable d'après $p$

### B.2.c - Calculez la p-value de votre test. A quoi correspond-elle ?


### B.2.d -  Prendre la décision (rejet ou non) concernant l'hypothèse posée


### B.2.e - Le test que vous venez de construire s'appelle en réalité un test binomial. Trouvez une librairie qui l'implémente et comparez avec votre p-value.


### B.2.f - Réaliser ce même test pour tout l'enregistrement pour chaque intervalle de 100 mesures et retracer le signal avec une couleur différente pour les intervalles pour lesquels l'hypothèse $H_0$ a été rejetée. Ré-essayer avec $\alpha = 0.000001$.


### B.2.g - Estimer la proportion de faux positifs avec $\alpha = 0.05$ . Est-ce la valeur attendue, et pourquoi ?

### B.2.f - Extraire le son détecté et l'enregistrer dans un fichier .wav

Avec $n=100$, on se rend compte qu'il y a certains intervalles au milieu du son ou l'hypothèse nulle n'est pas rejetée (des faux négatifs donc). Plottez ces intervalles.

Avec un $n$ trop petit, on se retrouve avec le même problème que par la méthode du seuillage (B.3 itération 1) : on a soit trop de faux négatifs, soit trop de faux positifs. En effet comme $n=100$ représente $1/480$ème de seconde (puisque l'échantillonnage est de 48000Hz), il est tout à fait possible d'avoir des intervalles avec 100 amplitudes faibles au milieu d'un son.

Refaire le test pour des intervalles d'une seconde ($n = 48000$) avec $\alpha = 0.05$, replotter le résultat, et enregistrer l'intervalle de son détecté dans un fichier wav pour écouter le résultat de votre détection de son par test d'hypothèse. Vous pourrez utiliser la fonction `scipy.io.wavfile.write`.

### B.3 - Regarder la vidéo : https://www.youtube.com/watch?v=jy9b1HSqtSk puis répondre par vrai ou faux à chacune des questions suivantes

#### B.3.a - Pour chaque détection (ie $H_0$ rejetée) j’ai une probabilité 1 - pvalue qu’il s’agisse d’une “vraie détection”
#### B.3.b - En supposant $H_0$ vraie, j’ai une probabilité $\alpha$ de conclure le rejet de $H_0$

# Pour aller plus loin

Il existe de nombreux tests d'hypothèses bien établis pour tout un tas de situations courantes (comparaison de moyennes entre 2 échantilllons, test de correlation de deux échantillons, test de normalité d'un échantillon, ... ) qui sont décrits dans les chapitres II, III et IV du poly sur les tests d'hypothèses. Vous pouvez également regarder cet article qui les résume avec le code python correspondant :

https://machinelearningmastery.com/statistical-hypothesis-tests-in-python-cheat-sheet/

ou
https://towardsdatascience.com/demystifying-statistical-analysis-1-a-handy-cheat-sheet-b6229bf992cf

Et les notebook kaggle suivants:

https://www.kaggle.com/hamelg/python-for-data-24-hypothesis-testing

https://www.kaggle.com/hamelg/python-for-data-25-chi-squared-tests

https://www.kaggle.com/hamelg/python-for-data-26-anova

Sur l'inférence bayésienne pour remplacer un t-test :
https://www.youtube.com/watch?v=fhw1j1Ru2i0
http://nmouatta.blogspot.com/2016/08/the-bayesian-t-test-in-python_25.html